In [19]:
import math

import numpy as np
import pyphi

In [20]:
n_nodes = 6

In [21]:
n_nodes_max = 8
for n in range(1, n_nodes_max + 1):
    n_states = 2 ** n
    print(f"(n_nodes, n_states) = {n, n_states}")

(n_nodes, n_states) = (1, 2)
(n_nodes, n_states) = (2, 4)
(n_nodes, n_states) = (3, 8)
(n_nodes, n_states) = (4, 16)
(n_nodes, n_states) = (5, 32)
(n_nodes, n_states) = (6, 64)
(n_nodes, n_states) = (7, 128)
(n_nodes, n_states) = (8, 256)


In [22]:
def generate_states_with_binary_state(n_nodes, seed=42):
    """
    Generates a matrix of random probabilities and a binary state vector with reproducibility.
    Each row of the random probabilities matrix sums to 1.
    The state vector is binary (0 or 1).
    
    Parameters:
    - n_nodes: The number of nodes.
    - seed: The seed for the random number generator to ensure reproducibility.
    
    Returns:
    - random_probs: A (2^n_nodes, 2^n_nodes) numpy array where each row sums to 1.
    - binary_state: A tuple representing a binary (0 or 1) state with n_nodes elements.
    """
    np.random.seed(seed)
    
    matrix_size = 2 ** n_nodes
    
    # Generate random numbers for the matrix
    random_matrix = np.random.rand(matrix_size, matrix_size)
    # Normalize each row to sum to 1
    random_probs = random_matrix / random_matrix.sum(axis=1)[:, np.newaxis]
    
    # Generate the binary state vector
    binary_state_array = np.random.randint(2, size=(n_nodes, 1))
    binary_state = tuple(binary_state_array.flatten())
    
    return random_probs, binary_state

# Generate with binary state
tpm, state = generate_states_with_binary_state(n_nodes, seed=42)
# print(tpm.shape, state.shape)
print(tpm.shape, len(state))
# print(tpm)
# print("===========")
# print(state)

(64, 64) 6


In [23]:
sbn_tpm = pyphi.convert.state_by_state2state_by_node(tpm)
sbs_tpm = pyphi.convert.state_by_node2state_by_state(sbn_tpm)

print(sbn_tpm.shape, sbs_tpm.shape)

(2, 2, 2, 2, 2, 2, 6) (64, 64)


In [24]:
pyphi.config.PROGRESS_BARS = False
pyphi.config.PARALLEL = False
pyphi.config.SHORTCIRCUIT_SIA = False

# Defining a substrate
# node_labels = ("A", "B", "C")

# TPM in state-by-node format (see `help(pyphi.Network)` for information on TPM
# representations).
# tpm = np.array(
#     [
#         [1, 0, 0],
#         [0, 1, 0],
#         [1, 1, 1],
#         [0, 1, 1],
#         [0, 0, 0],
#         [1, 1, 0],
#         [0, 0, 1],
#         [1, 0, 1],
#     ]
# )

# The network's adjacency matrix. In its absence, PyPhi will assume all-to-all
# connectivity. PyPhi uses the "from-to" convention: a "1" in entry (i,j) means
# there is a directed edge from unit i to unit j.

# NOTE: If the adjacency matrix indicates a connection between units when there
# is no causal connection according to the TPM, the analysis will correctly
# determine that; however, if the matrix specifies the *lack* of a connection,
# PyPhi will assume no causal influence and short-circuit the analysis, even if
# the TPM implies that there is.
# connectivity_matrix = np.array([
#     [1, 1, 0,],  # A->A, A->B
#     [0, 1, 1,],  # B->B, B->C
#     [1, 1, 1,],  # C->A, C->B, C->C
# ])

# network = pyphi.Network(tpm, cm=connectivity_matrix, node_labels=node_labels)
# network = pyphi.Network(tpm)
network = pyphi.Network(sbs_tpm)
# network
# System Irreducibility Analysis: identifying complexes
## Intrinsicality: Define a candidate system
### A=ON, B=OFF, C=OFF (Abc in Figure 7).
# state = (1, 0, 0)

# subsystem_backward = pyphi.Subsystem(
#     network,
#     state,
#     nodes=node_labels,
#     backward_tpm=True
# )
subsystem_backward = pyphi.Subsystem(
    network,
    state,
    backward_tpm=True
)

# subsystem_forward = pyphi.Subsystem(
#     network,
#     state,
#     nodes=node_labels
# )
subsystem_forward = pyphi.Subsystem(
    network,
    state,
)

## Composition: Unfolding Φ-structure
### Cause-effect structure and big Φ
phi_structure = pyphi.new_big_phi.phi_structure(subsystem_forward)
# phi_structure

#### φ_s
print(f"φ_s = {phi_structure.sia.phi}")

#### Big Φ (sum of distinctions and relations' small φ)
print(f"  Φ = {phi_structure.big_phi}")

φ_s = -8.904744115994914e-05
  Φ = 3.771928248186974


In [25]:
"""
4 nodes: 5.8s
5 nodes: 2m31.9s
6 nodes: 133m19.7s
"""

'\n4 nodes: 5.8s\n5 nodes: 2m31.9s\n'